In [7]:
import numpy as np
import os
from joblib import delayed, Parallel
import time
from analysis_funcs import rhg_lattice_scale, apply_error_channel, ParallelTqdm, find_unit_cell
import networkx as nx
import matplotlib.pyplot as plt
from cluster_sim.app.grid import Grid
import pandas as pd


os.makedirs("./data", exist_ok=True)

shape = [40, 40, 40]
seed = 1

samples = 1

# Input vector for all jobs
jobs_input_vec = [(p, scale) for scale in range(1, 4) for p in np.linspace(0, 0.3, 30)]


In [8]:
def main(input_params):
    """
    Main function for parallel processing. Here, we:

    1. Create a grid of the given shape
    2. Apply the error channel to the grid
    3. Generate a RHG lattice
    4. Look at clusters of the RHG lattice
    5. Return the results

    Returns:
        - p: The input parameter for the simulation
        - percolates: The number of times percolation occurred (True: 1, False: 0)
        - times: The average time taken for the simulation
    """
    
    p, scale = input_params

    # Sanity check: check that this is equal to the move_list on the app
    # print(np.reshape(np.argwhere(removed_nodes == True), shape=-1))
    
    percolates = 0
    
    # store the outputs for 1 simulation
    data_out = []

    for i in range(samples):
        start = time.time()
        G = Grid(shape)
        removed_nodes = np.zeros(shape[0] * shape[1] * shape[2], dtype=bool)

        G, D, removed_nodes = apply_error_channel(p, seed + i, shape, removed_nodes, G)
        # Generate an RHG lattice out of G
        G, D, removed_nodes, offsets = rhg_lattice_scale(G, D, removed_nodes, shape, scale_factor=scale)

        cells_count = find_unit_cell(G, shape, offsets, scale_factor=scale)
        end = time.time()

        data_out.append({
        "sample": i, 
        "p": p,
        "times": end - start,
        "unit_cells": cells_count,
        "scale": scale,
        })

    return data_out

In [9]:
results = ParallelTqdm(n_jobs=-1)([delayed(main)(x) for x in jobs_input_vec])

results = [item for sublist in results for item in sublist]

df = pd.DataFrame(results)
df.to_csv("./data/test.csv", index=False)


100%|██████████| 90/90 [00:58<00:00,  1.55tasks/s]
